In [5]:
# data collection
import cv2
import os,sys
import numpy as np
import csv

name = input("Enter name")
choice  = input("1:Images from camera\n2:images from gallery\n")
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

Id = 0

if os.path.exists("SubjectDetails.csv"):
    num_rows= 0
    for row in open("SubjectDetails.csv"):
        print(num_rows)
        num_rows += 1
    Id = num_rows
else:
    Id = "1"
    
if choice == "1":
    # body to get images from camera
    camera = cv2.VideoCapture(0)
    sample_num = 0
    while True:
        ret,frame = camera.read()
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = detector.detectMultiScale(gray,1.2,5)
        if faces is not ():
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),1)
                cv2.putText(frame,str(sample_num),(10,90),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1)
                sample_num += 1
                cv2.imwrite("subject_images/"+name +"."+str(Id) +'.'+str(sample_num) + ".jpg",gray[y:y+h,x:x+w])
        cv2.imshow("frame",frame)
        if cv2.waitKey(100) & 0xFF == 27:
            break
    camera.release()
    cv2.destroyAllWindows()
    
elif choice == "2":
    path = "steve jobs/"
    images_paths = [os.path.join(path,f) for f in os.listdir(path)]
    sample_num = 0
    for image in images_paths:
        image = cv2.imread(image)
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        faces = detector.detectMultiScale(gray,1.2,5)
        if faces is not ():
            for (x,y,w,h) in faces:
                sample_num += 1
                cv2.imwrite("subject_images/"+name +"."+str(Id) +'.'+str(sample_num) + ".jpg",gray[y:y+h,x:x+w])        
else:
    print("invalid value")
    
row = [name,Id]  
if os.path.exists("SubjectDetails.csv"):
    with open("SubjectDetails.csv",'a+',newline = '') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(row)
        csvfile.close()
else:
    with open("SubjectDetails.csv",'w',newline = '') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['name','Id'])
        writer.writerow(row)
        csvfile.close()

Enter name steve job
1:Images from camera
2:images from gallery
 2


0
1


In [24]:
# Training images

import os
import cv2
import numpy as np
from PIL import Image
import tqdm
import time

def getImagesAndLabels(path):
    imagePaths=[os.path.join(path,f) for f in os.listdir(path)]
    faces=[]
    Ids=[]

    for imagePath in imagePaths:
        pilImage = Image.open(imagePath).convert('L')
        imageNp=np.array(pilImage,'uint8')
        Id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces.append(imageNp)
        Ids.append(Id)
    return faces,Ids

def TrainImages():
    if os.path.exists("model.yml"):
        recognizer = cv2.face_LBPHFaceRecognizer.create()
        recognizer.read("model.yml")
    else:
        recognizer = cv2.face_LBPHFaceRecognizer.create()
        
    faces,Id = getImagesAndLabels("Subject_Images/")
# for training images multiple epocs uncomment below code
#     for i in tqdm.tqdm(range(50)):
#         recognizer.train(faces, np.array(Id))
#     print(Id)
    recognizer.train(faces, np.array(Id))
    recognizer.save("model.yml")
TrainImages()
res = "Image Trained"
print(res)

Image Trained


In [8]:
import cv2

image = cv2.imread("ste")
image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

recognizer = cv2.face_LBPHFaceRecognizer.create()
recognizer.read("model.yml")
Id,conf = recognizer.predict(image)
print(Id)

2


In [12]:
import cv2
import pandas as pd

recognizer = cv2.face_LBPHFaceRecognizer.create()
recognizer.read("model.yml")
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
df=pd.read_csv("SubjectDetails.csv")
camera = cv2.VideoCapture(0)

while True:
    ret,frame = camera.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    if ret:
        faces = detector.detectMultiScale(gray,1.3,5)
        if faces is not ():
            
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),1)
                Id,conf = recognizer.predict(gray[y:y+h,x:x+w])
                if (conf > 50):
                    name = df.loc[df['Id'] == Id]['name'].values
                    cv2.putText(frame,str(name),(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1)
                else:
                    cv2.putText(frame,"face unknown",(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1)
                cv2.imshow("frame",frame)
    else:
        print("face not found")
    if cv2.waitKey(100) & 0xFF == 27:
        break
camera.release()
cv2.destroyAllWindows()

In [18]:
import cv2
import pandas as pd

recognizer = cv2.face_LBPHFaceRecognizer.create()
recognizer.read("model.yml")
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
df = pd.read_csv("SubjectDetails.csv")

camera = cv2.VideoCapture(0)

while True:
    ret,frame = camera.read()
    if ret:
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = detector.detectMultiScale(gray,1.2,5)
        if faces is not ():    
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),1)
                Id,conf = recognizer.predict(gray[y:y+h,x:x+w])
                if (conf > 50):
                    name = df.loc[df['Id'] == Id]['name'].values
                    cv2.putText(frame,str(name),(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1)
                else:
                    cv2.putText(frame,"face unknown",(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1)
                cv2.imshow("frame",frame)
        else:
            print("no face found")
    else:
        print("no frame fetch")
    if cv2.waitKey(100) & 0xFF == 27:
        break
camera.release()
cv2.destroyAllWindows()



no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
